In [1]:
# LSTM for sequence classification in the IMDB dataset
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
import numpy as np

C:\Users\kmura\AppData\Local\Programs\Python\Python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Instructions for updating:
Use the retry module or similar alternatives.


In [5]:
import numpy as np
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    f = open(gloveFile,'r', encoding='utf8')
    model = {}
    for line in f:
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.asarray(splitLine[1:], dtype='float32')
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model

glove_model=loadGloveModel("D:/MyCodes/EMBEDDINGS/glove.840B.300d.txt")



Loading Glove Model
Done. 2196016  words loaded!


In [7]:
import pickle
with open("D:/MyCodes/EMBEDDINGS/glove_840B_300d.pickle","wb") as fh:
    pickle.dump(glove_model,fh)
    

In [9]:
import pickle
with open("D:/MyCodes/EMBEDDINGS/glove_840B_300d.pickle","rb") as fh:
    embed_dict=pickle.load(fh)


In [10]:
# Create embedding matrix

vocab=list(embed_dict.keys())




In [11]:
vocab.insert(0,"__unk__")


In [12]:
rev_vocab=dict((v,k) for k,v in enumerate(vocab))

In [13]:
with open("D:/SEM 7/Core Project/Dataset/Datasets/Classification_task/classification_clusters.pickle","rb") as fh:
    total_data=pickle.load(fh)

In [14]:
def create_labels(data):
    
    
    targets=[]
    
    for ques in data:
        if ques[2].lower() not in targets:
            targets.append(ques[2].lower())
            
    return targets
    
    

In [15]:
uniq_targets=create_labels(total_data)
rev_targets=dict((v,k) for k,v in enumerate(uniq_targets))

In [19]:

def encode_data(dictionary1,dictionary2,data):
    
    if dictionary1 ==None or dictionary2==None:
        print("Both dictionaries are not passed")
        return 
    
    encoded_totl_data=[]
    
    for ques in data:
        
        if "?" in ques[0]:
            ques[0]=ques[0].replace("?"," ?")

        
        encoded_ques=[dictionary1.get(wrd.lower(),) for wrd in (ques[0].split(" "))]  # Tokenizing and encoding
        encoded_targ=[dictionary2[ques[2].lower()]] # Encoding targets
        
        encoded_totl_data.append([encoded_ques,encoded_targ])
        
    return encoded_totl_data
    
    

In [20]:
encoded_total_data= encode_data(rev_vocab,rev_targets,total_data)
# try:
#     encoded_total_data= encode_data(rev_vocab,rev_targets,total_data)
# except Exception as e:
#     print("error is ",e)
#     encoded_total_data=[]

In [ ]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = 
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))